# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [1]:
import os
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [8]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 자모 단위 FASTTEXT

In [9]:
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install konlpy

from konlpy.tag import Mecab
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 4.09 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-15 08:40:37--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucke

In [18]:
kor_begin = 44032
kor_end = 55203
chosung_base = 588
jungsung_base = 28
jaum_begin = 12593
jaum_end = 12622
moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']


## 제4항 파열음 표기에는 된소리를 쓰지 않는 것을 원칙으로 한다.
## 된소리의 경우 id값에서 -1을 해주면 예사소리가 된다. 

## 거센소리로 통일시키는게 낫다는 판단이 들었다.

## 카라멜마끼아또 -> 카라멜마기아도 (X)

loanword_cho_list = ['ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ']

def loanword_cho(cho):
  if cho in loanword_cho_list:
    if cho == loanword_cho_list[0]:
      cho = 'ㅋ'
    elif cho == loanword_cho_list[1]:
      cho = 'ㅌ'
    elif cho == loanword_cho_list[2]:
      cho = 'ㅍ'
    elif cho == loanword_cho_list[3]:
      cho = 'ㅅ' ## ㅅ의 경우, 거센소리가 없으므로 예사소리로 통일시킨다.
    elif cho == loanword_cho_list[4]:
      cho = 'ㅊ'
  
  return cho

## 모음의 일반화, 의미가 통한다고 가정하고 2개가 합쳐진 모음은 하나로 뭉침
def loanword_jung(moum):
  if moum in ['ㅐ']:
    moum = 'ㅏ'
  elif moum in ['ㅒ']:
    moum = 'ㅑ'
  elif moum in ['ㅔ']:
    moum = 'ㅓ'
  elif moum in ['ㅖ']:
    moum = 'ㅕ'
  elif moum in ['ㅘ', 'ㅙ', 'ㅚ']:
    moum = 'ㅗ'
  elif moum in ['ㅝ', 'ㅞ', 'ㅟ']:
    moum = 'ㅜ'
  elif moum in ['ㅢ']:
    moum = 'ㅡ'
  
  return moum


## 제3항 받침에는 ‘ㄱ, ㄴ, ㄹ, ㅁ, ㅂ, ㅅ, ㅇ’만을 쓴다.
def loanword_jong(jamo):
  if jamo in ['ㄲ', 'ㄳ', 'ㄺ', 'ㅋ']:
    jamo = 'ㄱ'
  elif jamo in ['ㄵ', 'ㄶ']:
    jamo = 'ㄴ'
  elif jamo in ['ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ']:
    jamo = 'ㄹ'
  elif jamo in ['ㅄ', 'ㄶ','ㅍ']:
    jamo = 'ㅂ'
  elif jamo in ['ㄷ','ㅆ','ㅈ','ㅊ','ㅌ','ㅎ']:
    jamo = 'ㅅ'
  
  return jamo

In [ ]:
print(ord('ㄱ'))

12593


In [ ]:
print(chr(12593))

ㄱ


In [ ]:
print(ord('ㄲ'))

12594


In [ ]:
print(ord('ㄷ'))

12599


In [ ]:
print(ord('ㄸ'))

12600


In [19]:
import hgtk
from tqdm import tqdm
from konlpy.tag import Mecab

def word_to_jamo(token):
    def to_special_token(jamo):
      if not jamo:
        return '-'
      else:
        return jamo
    decomposed_token = ''

    for char in token:
        try:
        # char( 음 절 ) 을 초 성 , 중 성 , 종 성 으 로 분 리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자 모 가 빈 문 자 일 경 우 특 수 문 자 - 로 대 체
            cho = to_special_token(cho)
            cho = loanword_cho(cho)
            jung = to_special_token(jung)
            jung = loanword_jung(jung)
            jong = to_special_token(jong)
            jong = loanword_jong(jong)
            decomposed_token = decomposed_token + cho + jung + jong

        # 만 약 char( 음 절 ) 이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char

    # 단 어 토 큰 의 자 모 단 위 분 리 결 과 를 추 가
    return decomposed_token


mecab = Mecab()

def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [20]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("/gdrive/MyDrive/2023beaver/real_finish.csv")
df = df.dropna()

train_data, test_data = train_test_split(df, test_size=0.20, random_state=2)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=2)

In [ ]:
df['title'][0]

'국내산 100% 전라도 배추 김치 포기 김장김치 주문'

In [ ]:
word_to_jamo(df['title'][0])

'ㄱㅜㄱㄴㅐ-ㅅㅏㄴ ㅂㅐ-ㅊㅜ- ㄱㅣㅁㅊㅣ- ㅍㅗ-ㄱㅣ- ㄱㅣㅁㅈㅏㅇㄱㅣㅁㅊㅣ- ㅈㅜ-ㅁㅜㄴ'

In [ ]:
word_to_jamo("찜닭")

'ㅈㅣㅁㄷㅏㄱ'

In [ ]:
chr(ord('ㅉ') - 1)

'ㅈ'

In [ ]:
print(tokenize_by_jamo(df['title'][0]))

['ㄱㅜㄱㄴㅐ-ㅅㅏㄴ', '100', '%', 'ㅈㅓㄴㄹㅏ-ㄷㅗ-', 'ㅂㅐ-ㅊㅜ-', 'ㄱㅣㅁㅊㅣ-', 'ㅍㅗ-ㄱㅣ-', 'ㄱㅣㅁㅈㅏㅇ', 'ㄱㅣㅁㅊㅣ-', 'ㅈㅜ-ㅁㅜㄴ']


In [21]:
def tokenized(data):
  tokenized_data=[]

  for sample in tqdm(data['title'].to_list()):
      tokenzied_sample = tokenize_by_jamo(sample) # 자 소 단 위 토 큰 화
      tokenized_data.append(tokenzied_sample)

  return tokenized_data

In [ ]:
print(tokenized_data[0])

['ㄱㅜㄱㄴㅐ-ㅅㅏㄴ', '100', '%', 'ㅈㅓㄴㄹㅏ-ㄷㅗ-', 'ㅂㅐ-ㅊㅜ-', 'ㄱㅣㅁㅊㅣ-', 'ㅍㅗ-ㄱㅣ-', 'ㄱㅣㅁㅈㅏㅇ', 'ㄱㅣㅁㅊㅣ-', 'ㅈㅜ-ㅁㅜㄴ']


In [22]:
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

# 1. 초 기 입 력
# jamo_sequence = ' ﾤ ￂ ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '

    while index < len(jamo_sequence):
    # 문 자 가 한 글 ( 정 상 적 인 자 모 ) 이 아 닐 경 우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1

    # 문 자 가 정 상 적 인 자 모 라 면 초 성 , 중 성 , 종 성 을 하 나 의 토 큰 으 로 간 주 .
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3

# 2. 자 모 단 위 토 큰 화 완 료
# tokenized_jamo : [' ﾤ ￂ ﾱ ', ' ﾧ ￌ ﾷ ', ' ﾵ ￃ ﾷ ']

        word = ''
        try:
            for jamo in tokenized_jamo:
            # 초 성 , 중 성 , 종 성 의 묶 음 으 로 추 정 되 는 경 우
                if len(jamo) == 3:
                    if jamo[2] == "-":
                    # 종 성 이 존 재 하 지 않 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1])
                    else:
                # 종 성 이 존 재 하 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
                # 한 글 이 아 닌 경 우
                else:
                    word = word + jamo

            # 복 원 중 (hgtk.letter.compose) 에 러 발 생 시 초 기 입 력 리 턴 .
            # 복 원 이 불 가 능 한 경 우 예 시 ) ' ﾤ ! ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                return jamo_sequence

        # 3. 단 어 로 복 원 완 료
        # word : ' 남 동 생 '

    return word

In [ ]:
jamo_to_word(tokenized_data[0][0])

'국내산'

# 학습데이터 구축
학습데이터를 만듭니다.

In [23]:
train_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [24]:
train_token = tokenized(train_data)
val_token = tokenized(val_data)
test_token = tokenized(test_data)

100%|██████████| 41076/41076 [00:06<00:00, 5953.35it/s]


In [25]:
with open('train_data.txt', 'w') as out:
    for i in tqdm(range(len(train_token)), unit=' line'):
          out.write("__label__" + train_data['category'][i] + "\t" + ' '.join(train_token[i]) + '\n')

with open('val_data.txt', 'w') as out:
    for i in tqdm(range(len(val_token)), unit=' line'):
        out.write("__label__" + val_data['category'][i] + "\t" + ' '.join(val_token[i]) + '\n')

with open('test_data.txt', 'w') as out:
    for i in tqdm(range(len(test_token)), unit=' line'):
        out.write("__label__" + test_data['category'][i] + "\t" + ' '.join(test_token[i]) + '\n')

100%|██████████| 41076/41076 [00:00<00:00, 112868.31 line/s]


# 학습


In [26]:
import fasttext

model = fasttext.train_supervised(input='train_data.txt', autotuneValidationFile='val_data.txt')

In [27]:
model.predict(word_to_jamo(""))

(('__label__과자/떡/베이커리',), array([0.82904822]))

In [15]:
word_to_jamo("찜닭")

'ㅈㅣㅁㄷㅏㄱ'

찜닭 짐닭 짐닥
카라멜마끼아또
카라멜마키아또
카라멜마키아토


1. 모든 된소리의 예사소리화

-> 0.9100 정도로 그다지 차이가 없음

2. 종성 일반화

-> 0.9095 정도로 그다지 차이가 없음. 소폭 정확도는 떨어짐

3. 모음 일반화

-> 0.9099

결론

- 단어를 줄였음에도 불구하고, 평가 지표에서 큰 차이가 생기지 않았다. 불필요한 특성을 제거해서 속도 측면에서의 성능을 향상시키는 효과를 기대해볼 수도 있을거라고 생각했다. 

# 테스트
결과는 (샘플, 정확도, 재현율)

In [14]:
model.test("test_data.txt")

(41076, 0.9099961047813808, 0.9099961047813808)